In [ ]:
# Part2_Graphs_and_Flows.ipynb

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
import pandas as pd

# === Basic Graph Structure ===

# Define a simple state
class BasicState(TypedDict):
    """Basic state for demonstration"""
    data: str
    processed: bool
    steps_completed: list

# Simple processing nodes
def load_data(state: BasicState):
    """Load data into the state"""
    return {
        "data": "Sample data loaded",
        "steps_completed": state.get("steps_completed", []) + ["load"]
    }

def process_data(state: BasicState):
    """Process the data in the state"""
    return {
        "processed": True,
        "steps_completed": state.get("steps_completed", []) + ["process"]
    }

# Create and run basic graph
def create_basic_graph():
    """Create a simple linear graph"""
    graph = StateGraph(BasicState)
    
    # Add nodes
    graph.add_node("load", load_data)
    graph.add_node("process", process_data)
    
    # Add edge
    graph.add_edge("load", "process")
    graph.add_edge("process", END)
    
    # Compile
    return graph.compile()

# === Conditional Routing ===

class RoutingState(TypedDict):
    """State for conditional routing"""
    data_type: str
    data: str
    result: str
    steps: list

def analyze_numbers(state: RoutingState):
    """Analyze numerical data"""
    return {
        "result": "Numerical analysis complete",
        "steps": state.get("steps", []) + ["numerical"]
    }

def analyze_text(state: RoutingState):
    """Analyze text data"""
    return {
        "result": "Text analysis complete",
        "steps": state.get("steps", []) + ["text"]
    }

def route_by_type(state: RoutingState):
    """Route based on data type"""
    return "numbers" if state["data_type"] == "numerical" else "text"

def create_routing_graph():
    """Create a graph with conditional routing"""
    graph = StateGraph(RoutingState)
    
    # Add nodes
    graph.add_node("numbers", analyze_numbers)
    graph.add_node("text", analyze_text)
    
    # Add conditional routing
    graph.add_conditional_edges(
        "start",
        route_by_type,
        {
            "numbers": "numbers",
            "text": "text"
        }
    )
    
    # Add end edges
    graph.add_edge("numbers", END)
    graph.add_edge("text", END)
    
    return graph.compile()

# === Loop Implementation ===

class LoopState(TypedDict):
    """State for loop processing"""
    data: list
    processed: list
    complete: bool

def process_chunk(state: LoopState):
    """Process a chunk of data"""
    if not state.get("data"):
        return {"complete": True}
    
    # Process first item
    current = state["data"][0]
    remaining = state["data"][1:]
    processed = state.get("processed", []) + [f"Processed: {current}"]
    
    return {
        "data": remaining,
        "processed": processed,
        "complete": False
    }

def check_completion(state: LoopState):
    """Check if processing is complete"""
    return "end" if state["complete"] else "process"

def create_loop_graph():
    """Create a graph with a processing loop"""
    graph = StateGraph(LoopState)
    
    # Add processing node
    graph.add_node("process", process_chunk)
    
    # Add conditional edges for loop
    graph.add_conditional_edges(
        "process",
        check_completion,
        {
            "process": "process",
            "end": END
        }
    )
    
    return graph.compile()

# === Exercise Templates ===

def exercise_1():
    """
    Exercise 1: Create a linear graph that:
    1. Loads data
    2. Cleans data
    3. Analyzes data
    """
    # Your code here
    pass

def exercise_2():
    """
    Exercise 2: Create a router that:
    - Handles numerical data differently from text data
    - Applies appropriate processing for each type
    """
    # Your code here
    pass

def exercise_3():
    """
    Exercise 3: Create a loop that:
    - Processes data in chunks
    - Tracks progress
    - Handles completion
    """
    # Your code here
    pass

# === Testing ===

if __name__ == "__main__":
    # Test basic graph
    print("Testing basic graph...")
    basic_graph = create_basic_graph()
    basic_result = basic_graph.invoke({"data": "", "processed": False, "steps_completed": []})
    print("Basic graph result:", basic_result)
    
    # Test routing graph
    print("\nTesting routing graph...")
    routing_graph = create_routing_graph()
    routing_result = routing_graph.invoke({"data_type": "numerical", "data": "", "result": "", "steps": []})
    print("Routing graph result:", routing_result)
    
    # Test loop graph
    print("\nTesting loop graph...")
    loop_graph = create_loop_graph()
    loop_result = loop_graph.invoke({"data": ["item1", "item2", "item3"], "processed": [], "complete": False})
    print("Loop graph result:", loop_result)